In [1]:
# valuation_date時におけるcodeで指定した銘柄の個別銘柄分析
# SBI証券のモバイルの銘柄分析を参考に作成

In [2]:
### パラメータ設定セル
from datetime import date

# パラメータをセット
# code: 証券コード
# valuation_date: 過去にさかのぼって評価日をdatetime.dateでセット可能。デフォルト値は今日(date.today())
code = 8316
valuation_date = date.today()
# valuation_date = date(2023, 11, 29)

In [3]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import PricelistPl, FinancequotePl, KessanPl, KessanFig, read_data, get_df_latest_yearly_performance, get_latest_stockprice, print_finance_quote
from lib_dataprocess import get_fig_expected_performance_progress_rate_pycharts, MeigaralistPl, get_fig_actual_performance_progress_rate_pycharts, ShikihoPl
import polars as pl

# dataの読み込み
fp = DATA_DIR/"raw_pricelist.parquet"
df = read_data(fp)
PPL = PricelistPl(df)

fp = DATA_DIR/"finance_quote.parquet"
df = read_data(fp)
FPL = FinancequotePl(df)

fp = DATA_DIR/"kessan.parquet"
df = read_data(fp)
KPL = KessanPl(df)

fp = DATA_DIR/"meigaralist.parquet"
df = read_data(fp)
MPL = MeigaralistPl(df)

fp = DATA_DIR/"shikiho.parquet"
df = read_data(fp)
SPL = ShikihoPl(df)

In [4]:
### 最新情報
# 指定したcodeの指定した日における株価と各種ファンダメンタルズデータをまとめて標準出力する
# pricelist_dfは、raw_pricelistかreviced_pricelistかケースに応じて使い分ける。
# def print_finance_quote(
#        pricelist_df: pl.DataFrame,
#        finance_quote_df: pl.DataFrame,
#        code: int, 
#        valuation_date: date=date.today()
#    ) -> None:
print_finance_quote(PPL.df, FPL.df, code, valuation_date=valuation_date)
print()
SPL.print_latest_stock_df(code, valuation_date)

三井住友 FG(8316)の銘柄情報

終値: 3686.0円(2024年11月29日)
予想配当利回り: 3.26%(2024年11月29日)
予想PER: 12.47倍(2024年11月29日)
実績PBR: 0.98倍(2024年11月29日)
自己資本比率: 5.0%(2024年11月29日)
予想ROE: 7.84%(2024年11月29日)
予想ROA: 0.39%(2024年11月29日)

8316(三井住友 FG)の2024年09月13日発行四季報データ
【連続最高益】
  国内法人軸に貸出残高漸増。国内金利上昇で利ザヤ改善する。カード決済など非金融も堅調。上期軸に株売却益。航空機リース保険金消えるが、与信費用横ばい圏で経常増益続く。最高純益。増配。
【マネーフォワード】
  三井住友カードが資本提携、合弁会社設立し49%出資。当社金融サービスと家計管理機能統合し新事業創出へ。欧州でプライベートファンド組成。


In [5]:
### 業績推移

# KessanFig(
#    code: int, 
#    settlement_type: Literal["通期", "四半期"], 
#    output_target: str = "jupyter",
#    start_settlement_date: date = date(1900, 1, 1),
#    end_settlement_date: date = date(2999, 12, 31)
#)

KFIG = KessanFig(code, "通期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

RuntimeError: BindingsError: "Could not determine output type"

In [ ]:
### 通期業績、業績予想
# def get_df_latest_yearly_performance(code: int, valuation_date: date=date.today()) -> pl.DataFrame:
df = get_df_latest_yearly_performance(code)
df

In [ ]:
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

In [ ]:
# codeで指定した銘柄のevaluation_dateで指定した時点での最新の年度決算予想に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# from plotly.graph_objects import Figure
# def get_fig_expected_performance_progress_rate_pycharts(code: int, evaluation_date: date=date.today()) -> Figure:

fig1 = get_fig_expected_performance_progress_rate_pycharts(code, valuation_date)
fig1.show()

In [ ]:
last_year_valuation_date = date(valuation_date.year-1, valuation_date.month, valuation_date.day)

# codeで指定した銘柄のevaluation_dateで指定した時点での最新の四半期決算発表に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# valuation_dateで指定する日において、当年度の本決算が発表されていない日を指定した場合は前期末第4四半期の進捗率を表示するので、進捗率100%で表示される。
fig2 = get_fig_actual_performance_progress_rate_pycharts(code, last_year_valuation_date, KPL.df, MPL.df)
fig2.show()